Testing of Landsat-landstats for Kenya

In [1]:
%matplotlib inline
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd
import numpy as np

/Users/patrick/miniconda3/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


ImportError: No module named 'geopandas'

In [ ]:
# loading data
df = pd.DataFrame.from_csv('../data/estimates/estimates_after_join.csv')
df = df.dropna(axis=0)
df.tail()

In [ ]:
# we only need a subset of this data
df_subloc = df[['estimates', 'area', 'location_id', 
                'sublocation_id', 'pop']]
# renaming
df_subloc = df_subloc.rename(
    columns = {'pop': 'pop_sublocation_actual',
              'estimates' : 'density_subloc_est'})

The rows are single 32 x 32 satellite images. 

The convnet predicts the density for each image. This prediction is column 'estimates'.

In [ ]:
df_subloc = df_subloc.groupby(['sublocation_id']).mean()
df_subloc.reset_index(level=0, inplace=True)
df_subloc['pop_sublocation_est'] = df_subloc.density_subloc_est * df_subloc.area

In [ ]:
df_subloc.tail()

Looking at the bottom of the table, it seems like the population estimates are off by an order of 1000. However, it's pretty accurate! Let's look at the top

In [ ]:
df_subloc.head()

Our estimates are off by an order of 1000, but it's much less accurate.

The rest of the literature simply takes census data at some level (say Location) and uses weights to distribute across lower levers (e.g. Sublocation). So at the Location level, the population matches the Census. The test is whether the population matches the census at the Sublocation level.


A more ambitious project is simply estimating population at the Sublocation level. Let's get some metrics to do this.

In [ ]:
# Root mean square error
df_subloc['error_subloc'] = df_subloc.pop_sublocation_est - df_subloc.pop_sublocation_actual
df_subloc['sq_error_subloc'] = df_subloc.error_subloc**2
rmse = np.sqrt(df_subloc.sq_error_subloc.mean())
print 'The root mean squared error is: ', rmse

# %RMSE: RMSE / mean(population_subloc)
pc_rmse = 100 * rmse / df_subloc.pop_sublocation_actual.mean()
print 'The root mean squared percentage error is: ', pc_rmse

# Mean absolute error
df_subloc['abs_error_subloc'] = abs(df_subloc.pop_sublocation_est - df_subloc.pop_sublocation_actual)
mae = df_subloc.abs_error_subloc.mean()
print 'The mean absolute error is: ', mae

Wow! The RMSE and %RMSE are of the same order of magnitude as the literature _with populations matched at the location level as discussed above._ Further, the MAE is an order of magnitude greater. This table is taken from Stevens et al 2015. It's the state of the art. Their work is under rows 'RF'.

In [ ]:
Image(filename='stevens_table_2.png')

In [ ]:
# how about a scatter plot
plt.scatter(df_subloc.pop_sublocation_est, df_subloc.pop_sublocation_actual)

So it looks less good here. That's strangely comforting.

Now, let's use the method the literature does.

In [ ]:
# getting location level populations from sublocation data
df_loc = df_subloc[['pop_sublocation_actual', 'pop_sublocation_est', 
                    'location_id']].groupby('location_id').sum()
df_loc.reset_index(level=0, inplace=True)
df_loc = df_loc.rename(columns = {'pop_sublocation_actual': 'pop_location_actual',
                                 'pop_sublocation_est' : 'pop_location_est'})

# merge with sublocation data
df_final = pd.merge(df_subloc, df_loc, on = 'location_id', how = 'left')

# generating weights within a sublocation
df_final['weights']= df_final.pop_sublocation_est / df_final.pop_location_est

# distributing population at the location level by these weights
df_final['pop_sublocation_est_final'] = df_final.weights * df_final.pop_location_actual

In [ ]:
# Root mean square error
df_final['error_subloc_final'] = df_final.pop_sublocation_est_final - df_final.pop_sublocation_actual
df_final['sq_error_subloc_final'] = df_final.error_subloc_final**2
rmse_final = np.sqrt(df_final.sq_error_subloc_final.mean())
print 'The root mean squared error is: ', rmse_final

# %RMSE: RMSE / mean(population_subloc)
pc_rmse_final = 100 * rmse_final / df_final.pop_sublocation_actual.mean()
print 'The root mean squared percentage error is: ', pc_rmse_final

# Mean absolute error
df_final['abs_error_subloc_final'] = abs(df_final.pop_sublocation_est_final - df_final.pop_sublocation_actual)
mae_final = df_final.abs_error_subloc_final.mean()
print 'The mean absolute error is: ', mae_final

In [ ]:
sns.set_style("white")
plt.scatter(df_final.pop_sublocation_est_final, df_final.pop_sublocation_actual)
plt.axis([0, 120000, 0, 120000])
sns.axlabel('Estimated population', 'Actual population')
plt.savefig('scatter.png', bbox_inches='tight', dpi=300)

Wow. Wow.

** Mapping **

In [ ]:
df_geo = gpd.GeoDataFrame.from_file('../data/shapefiles/ke/sublocations/ke_1999.shp')
df_geo = df_geo.rename(columns={'SLID': 'sublocation_id'})

In [ ]:
df_geo_sl = pd.merge(df_final, df_geo, on = 'sublocation_id', how = 'inner')
df_geo_sl = gpd.GeoDataFrame(df_geo_sl)
df_geo_sl.tail()

Plotting maps.

Taken from: http://nbviewer.jupyter.org/gist/jorisvandenbossche/d4e6efedfa1e4e91ab65

In [ ]:
import numpy as np
from geopandas.plotting import (plot_linestring, plot_point, norm_cmap)


def plot_dataframe(s, column=None, colormap=None, alpha=0.5,
                   categorical=False, legend=False, axes=None, scheme=None,
                   k=5, linewidth=1):
    """ Plot a GeoDataFrame

        Generate a plot of a GeoDataFrame with matplotlib.  If a
        column is specified, the plot coloring will be based on values
        in that column.  Otherwise, a categorical plot of the
        geometries in the `geometry` column will be generated.

        Parameters
        ----------

        GeoDataFrame
            The GeoDataFrame to be plotted.  Currently Polygon,
            MultiPolygon, LineString, MultiLineString and Point
            geometries can be plotted.

        column : str (default None)
            The name of the column to be plotted.

        categorical : bool (default False)
            If False, colormap will reflect numerical values of the
            column being plotted.  For non-numerical columns (or if
            column=None), this will be set to True.

        colormap : str (default 'Set1')
            The name of a colormap recognized by matplotlib.

        alpha : float (default 0.5)
            Alpha value for polygon fill regions.  Has no effect for
            lines or points.

        legend : bool (default False)
            Plot a legend (Experimental; currently for categorical
            plots only)

        axes : matplotlib.pyplot.Artist (default None)
            axes on which to draw the plot

        scheme : pysal.esda.mapclassify.Map_Classifier
            Choropleth classification schemes

        k   : int (default 5)
            Number of classes (ignored if scheme is None)


        Returns
        -------

        matplotlib axes instance
    """
    import matplotlib.pyplot as plt
    from matplotlib.lines import Line2D
    from matplotlib.colors import Normalize
    from matplotlib import cm

    if column is None:
        return plot_series(s.geometry, colormap=colormap, alpha=alpha, axes=axes)
    else:
        if s[column].dtype is np.dtype('O'):
            categorical = True
        if categorical:
            if colormap is None:
                colormap = 'Set1'
            categories = list(set(s[column].values))
            categories.sort()
            valuemap = dict([(k, v) for (v, k) in enumerate(categories)])
            values = [valuemap[k] for k in s[column]]
        else:
            values = s[column]
        if scheme is not None:
            binning = __pysal_choro(values, scheme, k=k)
            values = binning.yb
            # set categorical to True for creating the legend
            categorical = True
            binedges = [binning.yb.min()] + binning.bins.tolist()
            categories = ['{0:.2f} - {1:.2f}'.format(binedges[i], binedges[i+1]) for i in range(len(binedges)-1)]
        cmap = norm_cmap(values, colormap, Normalize, cm)
        if axes == None:
            fig = plt.gcf()
            fig.add_subplot(111, aspect='equal')
            ax = plt.gca()
        else:
            ax = axes
        for geom, value in zip(s.geometry, values):
            if geom.type == 'Polygon' or geom.type == 'MultiPolygon':
                plot_multipolygon(ax, geom, facecolor=cmap.to_rgba(value), alpha=alpha, linewidth=linewidth)
            elif geom.type == 'LineString' or geom.type == 'MultiLineString':
                plot_multilinestring(ax, geom, color=cmap.to_rgba(value))
            # TODO: color point geometries
            elif geom.type == 'Point':
                plot_point(ax, geom, color=cmap.to_rgba(value))
        if legend:
            if categorical:
                patches = []
                for value, cat in enumerate(categories):
                    patches.append(Line2D([0], [0], linestyle="none",
                                          marker="o", alpha=alpha,
                                          markersize=10, markerfacecolor=cmap.to_rgba(value)))
                ax.legend(patches, categories, numpoints=1, loc='best')
            else:
                # TODO: show a colorbar
                raise NotImplementedError

    plt.draw()
    return ax

In [ ]:
df_geo_sl_prov = df_geo_sl[df_geo_sl.PROVID == 2.0]

In [ ]:
sns.set_style("white")
df_geo_sl_prov.plot(column='pop_sublocation_est_final', linewidth=0.25, figsize=(10, 10), scheme='Quantiles', k=5, colormap='winter', legend=True)
sns.axlabel('Longitude', 'Latitude')
plt.savefig('map_est.png', bbox_inches='tight', dpi=300)

In [ ]:
df_geo_sl_prov.plot(column='pop_sublocation_actual', linewidth=0.25, figsize=(10, 10), scheme='Quantiles', k=5, colormap='winter', legend=True)
sns.axlabel('Longitude', 'Latitude')
plt.savefig('map_actual.png', bbox_inches='tight', dpi=300)

In [ ]:
df_geo_sl_prov.plot(column='error_subloc_final', linewidth=0.25, figsize=(10, 10), scheme='Quantiles', k=5, colormap='seismic', legend=True)
sns.axlabel('Longitude', 'Latitude')
plt.savefig('map_error.png', bbox_inches='tight', dpi=300)